In [1]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string,one_hot
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer,MQOps
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore import Parameter, Tensor
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
# #Operator Pool


/opt/miniconda3/envs/Quantum/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/Quantum/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [2]:
unbound_opeartor_pool = \
["X0 X1 I2 I3 I4 I5 I6 I7", 
 "I0 X1 X2 I3 I4 I5 I6 I7",
 "I0 I1 X2 X3 I4 I5 I6 I7",
 "I0 I1 I2 X3 X4 I5 I6 I7",
 "I0 I1 I2 I3 X4 X5 I6 I7",
 "I0 I1 I2 I3 I4 I4 X6 I7",
 "Y0 Y1 I2 I3 I4 I4 I6 I7",
 "I0 Y1 Y2 I3 I4 I4 I6 I7",
 "I0 I1 Y2 Y3 I4 I4 I6 I7",
 "I0 I1 I2 Y3 Y4 I4 I6 I7",
 "I0 I1 I2 I3 Y4 Y5 I6 I7",
 "I0 I1 I2 I3 I4 Y5 Y6 I7",
 ]
bound_opeartor_pool = \
['ZYIXXXZX',
 'IZZYYXZI',
 'XIIXYIIZ',
 'IIIYYIYX',
 'IZZYIXZI',
 'ZZYYYYYY']

num_layer = 7
# 定义标准差和形状
stddev = 0.02
shape_parametized = len(unbound_opeartor_pool)
shape_unparametized = len(bound_opeartor_pool)
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)

nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)

np.random.seed(10)
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(shape_parametized)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(shape_parametized,shape_parametized+shape_unparametized)]

In [3]:
from DQAS_tool import Mindspore_ansatz2,sampling_from_structure,nmf_gradient,vag_nnp2


In [4]:
ansatz = Mindspore_ansatz2(Structure_p=stp,parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)

In [13]:
batch_size=256
ops_onehot = ops.OneHot(axis=-1)
tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))


In [14]:
batch_structure[0]
tmp[0]

array([ 6,  8,  1,  7, 14, 17, 12])

In [8]:
for each_layer in batch_structure[0]:
    # print(each_layer)
    print(ops.Argmax()(each_layer))

2
6
12
8
7
11
9


In [18]:
nnp.shape

(7, 12)

In [15]:
ansatz = Mindspore_ansatz2(Structure_p=batch_structure[0],parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)
ansatz.ansatz_params_name

['ansatz0', 'ansatz1', 'ansatz2', 'ansatz3']

In [16]:
g = vag_nnp2(Structure_params=batch_structure[0],Ansatz_params=nnp,paramerterized_pool=unbound_opeartor_pool,unparamerterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)

[Tensor(shape=[], dtype=Int32, value= 2), Tensor(shape=[], dtype=Int32, value= 10), Tensor(shape=[], dtype=Int32, value= 4), Tensor(shape=[], dtype=Int32, value= 11), Tensor(shape=[], dtype=Int32, value= 9), Tensor(shape=[], dtype=Int32, value= 7), Tensor(shape=[], dtype=Int32, value= 11)]
[0.04090865878616149, 0.025358561448930717, 0.0361049252974905, 0.042638790534355445, 0.03013244989482282, 0.022747611188208956, 0.03221753341251426]


ValueError: Weight init shape error, required (4, ), but get (7,).

In [ ]:
ansatz = Mindspore_ansatz2(Structure_p=batch_structure[3],parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)

gs = nmf_gradient(structures=stp,oh=batch_structure[0],num_layer=num_layer,size_pool=shape_parametized+shape_unparametized)
gs

In [ ]:
ansatz.ansatz_params_name

In [ ]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
from DQAS_tool import  DQASAnsatz_from_result,vag_nnp,nmf_gradient
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.05))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = False
# 设置超参数
epochs = 100
batch_size=256
# shape_nnp = (num_layer, shape_parametized)
# shape_stp = (num_layer, shape_parametized+7)
# nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
# stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
print('sdsd',stp.shape)
avcost1 = 0

ops_onehot = ops.OneHot(axis=-1)

avcost2 = avcost1

costl = []
tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
#print(batch_structure.shape)
# print(tmp,batch_structure)
loss_value = []
grad_nnps = []
grad_stps = []
    
for i in batch_structure:          
    infd, grad_nnp = vag_nnp(Structure_params=i,Ansatz_params=nnp,paramerterized_pool=unbound_opeartor_pool,unparamterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
    gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=shape_parametized+shape_unparametized)
    #print(infd,grad_nnp)
    loss_value.append(infd)
    grad_nnps.append(grad_nnp[0])
    grad_stps.append(gs)

infd = ops.stack(loss_value)
gnnp = ops.addn(grad_nnps)
gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
gstp_averge = ops.addn(gstp) / infd.shape[0]
avcost1 = sum(infd) / infd.shape[0]

    
print(gstp_averge)
 